In [ ]:
#FAISSと新しいembeddingモデルを試すだけの実装

In [9]:
!pip install -qU langchain-openai
import getpass
import os

os.environ["OPENAI_API_KEY"] ="YOUR_API_KEY"
from langchain_openai import OpenAIEmbeddings


In [28]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
in_embeds = embeddings.embed_documents(["引数保護機能の定義"])
target_texts = [
    "概要：SSP はスタックに対するバッファーオーバーフロー攻撃を検出するために、コンパイラにより生成される中間コードを拡張するようにコンパイラを拡張したものである。プログラムのソースコードになんら変更を加える必要はなく、コンパイル時に意識する必要はない。また実行コードに直接バッファーオーバーフロー攻撃の検出コードを挿入するわけではなく、中間言語として検出のためのコードを生成するため、さまざまな環境で SSP の技術を利用することが可能である。SSP によって生成されたプログラムは、スタックにおけるリターンアドレスへの不正な書き込みや変数・引数の不正な変更を検出することが可能である。SSP におけるバッファーオーバーフロー攻撃防御の仕組みは、基本的に StackGuard と同じである。つまりリターンアドレスの直前に「カナリア」（SSP では guard と呼ぶ）を挿入し、その値が関数の実行終了時に変更されているかどうかをチェックすることにより、バッファーオーバーフロー攻撃の発生を検出する。ただし、その実装に関しては、StackGuard とは若干違いがある。 特に StackGuard が実行コードレベルでバッファーオーバーフロー攻撃検出機能を挿入するのに対して、SSP では中間言語として検出機能を挿入するため、幅広いアーキテクチャのコンパイラをサポートすることができ、より一般的なバッファーオーバーフロー攻撃検出のための手法として利用が可能となる。またバッファーオーバーフロー攻撃としてスタックにおけるリターンアドレスの書き換えだけでなく、引数やフレームポインタの書き換え攻撃にも着目し、それらを保護するためのメカニズムを提供している点も異なっている。",
    "検出可能な脆弱性:SSP では以下の脆弱性を検出可能である。バッファーオーバーフロー攻撃に対する脆弱性（厳密にはスタックのオーバーフローによるリターンアドレスの不正な変更または、関数内の変数、フレームポインタの変更）",
    "SSP の機能詳細：SSP ではバッファーオーバーフロー攻撃によって書き換えられる可能性のある対象に対して、以下の 3 つの保護機能と通知機能を有している。1. リターンアドレス（フレームポインタ）の不正な変更検出機能- バッファーオーバーフロー攻撃によりスタック上のリターンアドレスやフレームポインタが書き換えられる攻撃を検出する機能2. 変数保護機能- バッファーオーバーフロー攻撃によって関数中で保持されている変数が書き換えられることを保護する機能3. 引数保護機能- バッファーオーバーフロー攻撃によって関数に引き渡される値が書き換えられることを保護する機能4. 通知機能- バッファーオーバーフロー攻撃検出の結果をログに書き出す機能",
    "1. リターンアドレス（フレームポインタ）の不正な変更検出機能:SSP では、バッファーオーバーフロー攻撃によるリターンアドレスやフレームポインタが書き換えられる攻撃を検出することが可能である。リターンアドレスやフレームポインタが書き換えられることを防ぐ機能を提供しているわけではない。SSP での基本的な検出方法SSP ではリターンアドレス、フレームポインタともに不正な変更を検知するために、フレームポインタの直前に guard と呼ばれるバッファーオーバーフロー攻撃を検知するための値を挿入する。ローカルの変数をオーバーフローさせてリターンアドレス、フレームポインタを修正する場合は、この guard を修正することになる。SSP では、関数から戻る際に、この guard が修正されていないかをチェックし、バッファーオーバーフロー攻撃の検出を行う。SSP におけるバッファーオーバーフロー攻撃の防御上記の基本的なコンセプト、および guard を利用し、SSP ではどのようにバッファーオーバーフロー攻撃を防御するか示す。ここでは、SSP が組み込まれたコードと組み込まれていないコードのアセンブリを示す。両者を比較することで SSP がどのようにバッファーオーバーフロー攻撃を防御しているのかをコードレベルで概観することができる。基本的には StackGuard と同じである。SSP では prologue に guard をフレームポインタの直前に配置するためのコードを挿入し、epilogue でスタック上に保存しておいた guard が変更されていないかをチェックし、変更されているならば、ログにメッセージを書き出して、終了するためのコードを埋め込む。実際の処理フローは以下のようになる。",
    "2. 変数保護機能:バッファーオーバーフロー攻撃によるリターンアドレスの変更検出のほかに SSP では関数内の変数がバッファーオーバーフロー攻撃によって書き換えられることを防ぐことが可能である。バッファーオーバーフロー攻撃は、リターンアドレスを書き換えるものとして知られているが、バッファーオーバーフローの対象となる変数とリターンアドレスの間にある変数は不正に書き換えられてしまうおそれがある。例えば、ここに他の関数へのポインタが格納された変数などがあれば、不正にプログラムの制御を奪われてしまう可能性がある。SSP ではこうした危険性からプログラムを保護するために、変数の保護機能を提供している。【関数ポインタを持った変数をねらった攻撃の防御】関数ポインタを持った変数をねらった攻撃に対して、SSP はスタックに格納される変数の順番を変更するという方法をとっている。これはバッファーオーバーフローによって変更されるおそれがある変数は、ターゲットとなる変数とリターンアドレスの間の変数であるという事実を利用して、スタックオーバーフローのターゲットとなる可能性のある文字列配列が、他の変数より前にスタックに積まれるようにするものである。ここでは、バッファーオーバーフローのターゲットとなる可能性のある buf 配列を関数内の他の変数の前にスタックに積んでいる。文字列は上位のアドレスへ伸びるので、リターンアドレスやフレームポインタは書き換えられても、変数を書き換えることはできない。これにより、たとえバッファーオーバーフローが発生しても、他の変数に極力被害が及ばないようにすることが可能となる。" ,
    "3. 引数保護機能:バッファーオーバーフロー攻撃によるリターンアドレスの変更検出のほかに、SSP では関数に引き渡される引数がバッファーオーバーフロー攻撃によって書き換えられることを防ぐことが可能である。前述した関数内の変数と同様に、引数も書き換えられることによって不正にプログラムの制御を奪われる可能性がある。例えば、引数に他の関数へのポインタが格納されている場合などは、不正にプログラムの制御を奪われてしまう。SSP ではこうした危険性からプログラムを保護するために、引数保護機能を提供している。【引数の保護】引数については、コンパイラの制約上、変数の保護のようにスタックに積む順番を自由に変更することはできない。そのため SSP では、引数のコピーを関数内に関数が呼び出された時点で作成しておき、コピーした関数を関数内では利用するものとしている。これにより、変数の保護と同じ仕組みを利用することが可能となる。",
    "4. 通知機能:SSP ではバッファーオーバーフローを検出した場合には、_stack_smashing_handler() 関数を呼び出す。この関数は、標準エラー出力、又は syslog に検出情報を書き出し、プログラムを停止する。実際には以下のようなエラーが syslog に書き出される。特にバッファーオーバーフローが発生した時のメモリ情報などはログとして出力されないが、バッファーオーバーフローが発生したプログラム、および関数が表示される。",
    "《利点》中間言語でバッファーオーバーフロー攻撃検出のためのコードを挿入するため、多様なアーキテクチャで SSP を利用可能である。コンパイラレベルで検出メカニズムを挿入するため、実行時のオーバーヘッドが低い。バッファーオーバーフロー攻撃を利用した複雑な攻撃（ポインタ変数の書き換えなど）にも対応可能である。コンパイラがスタック上にスタックポインタ上にフレームポインタを格納するコードを作らないもの（gcc –fomit-frame-pointer など）に対しても対応可能である。《欠点》アプリケーションをそれぞれソースコードからコンパイルしなおす必要がある。SSP の導入に対してコンパイラをコンパイルし直す必要がある。Libsafe ライブラリのようにバッファーオーバーフローを引き起こすことを防ぐことはできない。あくまでバッファーオーバーフローが発生して保護対象が不正に修正されたことを検出できるのみである。"]
embeds = []
for t in target_texts:
    embeds.append(embeddings.embed_query(t))
target_embeds = embeds

In [13]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.5 MB/s eta 0:00:00


In [18]:
print(len(target_embeds[0]))

3072


In [29]:
import faiss
import numpy as np

# インデックスの生成
index = faiss.IndexFlatL2(3072)
print(index.is_trained)
print(type(index))

# 対象テキストの追加
index.add(np.array(target_embeds).astype('float32'))
print(index.ntotal)

True
<class 'faiss.swigfaiss.IndexFlatL2'>
8


In [30]:
# 近傍探索の実行。
D, I = index.search(np.array(in_embeds).astype('float32'), 1)

# 確認
print(D)
print(I)
print(I[0][0])
print(target_texts[I[0][0]])

[[0.8272165]]
[[5]]
5
3. 引数保護機能:バッファーオーバーフロー攻撃によるリターンアドレスの変更検出のほかに、SSP では関数に引き渡される引数がバッファーオーバーフロー攻撃によって書き換えられることを防ぐことが可能である。前述した関数内の変数と同様に、引数も書き換えられることによって不正にプログラムの制御を奪われる可能性がある。例えば、引数に他の関数へのポインタが格納されている場合などは、不正にプログラムの制御を奪われてしまう。SSP ではこうした危険性からプログラムを保護するために、引数保護機能を提供している。【引数の保護】引数については、コンパイラの制約上、変数の保護のようにスタックに積む順番を自由に変更することはできない。そのため SSP では、引数のコピーを関数内に関数が呼び出された時点で作成しておき、コピーした関数を関数内では利用するものとしている。これにより、変数の保護と同じ仕組みを利用することが可能となる。
